In [1]:
%reload_ext autoreload
%load_ext dotenv 
%dotenv -o -v analysePython\\.env

from DbConnexion import DbConnexion



Db = DbConnexion(required_privileges={'SELECT'})

--- Initialisation de la connexion à la BDD ---
sae204now
Connexion à la base de données sae204now réussie !
--- vérification de la BDD ---
ping...
ping réussi
vérification des permissions...
Récupération des privilèges de l'utilisateur...
vérification des permissions réussie
Vérification des tables...
La table history_change existe
La table station_information existe
La table station_status existe


In [17]:
import folium
import mysql.connector
from mysql.connector import errorcode
from colors import Fore, Style
from cursorContext import cursorContext
import pandas as pd


query = """
SELECT i.stationcode, i.name, i.capacity, ST_X(coordonnees_geo) AS lon, ST_Y(coordonnees_geo) AS lat, 
i.nom_arrondissement_communes, s.date, s.is_installed, s.numdocksavailable, s.numbikesavailable, s.mechanical, s.ebike
    FROM station_information i 
    INNER JOIN station_status s ON s.stationcode = i.stationcode 
    WHERE s.date = ( 
      SELECT MAX(date) 
      FROM station_status 
      WHERE stationcode = i.stationcode 
    )
"""
stations = []
with cursorContext(Db.db) as cursor:
    try:
        cursor.execute(query, ())
    except Exception as e:
        print(Fore.RED + "Erreur lors de récupération des données : " + str(e) + Style.RESET_ALL)

    stations = cursor.fetchall()

data = pd.DataFrame(stations, columns=cursor.column_names)
display(data.head())

,stationcode,name,capacity,lon,lat,nom_arrondissement_communes,date,is_installed,numdocksavailable,numbikesavailable,mechanical,ebike
0,10001,René Boulanger - Lancry,62,2.360003,48.868543,Paris,2023-05-14 14:40:04,OUI,50,8,5,3
1,10003,Hauteville - Bonne Nouvelle,17,2.349650,48.870774,Paris,2023-05-14 14:40:04,OUI,11,5,2,3
2,10004,Metz - Faubourg Saint-Denis,48,2.353521,48.870893,Paris,2023-05-14 14:40:04,OUI,40,5,2,3
3,10005,Petites Ecuries - Faubourg Poissonnière,17,2.348389,48.873972,Paris,2023-05-14 14:40:04,OUI,15,1,0,1
4,10006,Petites Ecuries - Faubourg Saint-Denis,29,2.354215,48.872871,Paris,2023-05-14 14:40:04,OUI,23,5,1,4


In [18]:
print(len(data))

# filtrer donnée seulement paris
filter_data = data.loc[data['nom_arrondissement_communes'].str.lower() == 'paris']


m = folium.Map(location=[48.856, 2.352], zoom_start=12)

for index, row in filter_data.iterrows():
    folium.Marker([row['lat'], row['lon']], popup=row['name']).add_to(m)

m

1466
